In [3]:
!wget https://raw.githubusercontent.com/iraouiabdou/abdou_torch/main/abdou_torch.py
import abdou_torch
from abdou_torch import Tensor, Linear, MLP, WAdam

import torch
import torch.nn.functional as F
import numpy as np

--2025-12-01 17:46:09--  https://raw.githubusercontent.com/iraouiabdou/abdou_torch/main/abdou_torch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6557 (6.4K) [text/plain]
Saving to: ‘abdou_torch.py.1’

abdou_torch.py.1    100%[===================>]   6.40K  --.-KB/s    in 0s      

2025-12-01 17:46:09 (64.0 MB/s) - ‘abdou_torch.py.1’ saved [6557/6557]



In [2]:
def test_sanity_check():
    """
    Test a complex computation graph for both forward and backward pass.
    """
    x = Tensor(np.array([-4.0]))
    z = 2 * x + 2 + x
    q = z.relu() + z * x
    h = (z * z).relu()
    y = h + q + q * x
    y.backward()
    xat, yat = x, y



    x = torch.Tensor([-4.0]).double()
    x.requires_grad = True
    z = 2 * x + 2 + x
    q = F.leaky_relu(z, negative_slope=0.01) + z * x
    h = F.leaky_relu(z * z, negative_slope=0.01)
    y = h + q + q * x
    y.backward()
    xpt, ypt = x, y

    tol = 1e-9


    assert abs(yat.data.item() - ypt.data.item()) < tol, \
        f"Forward Pass Failed: Custom: {yat.data.item()}, PyTorch: {ypt.data.item()}"
    assert abs(xat.grad.item() - xpt.grad.item()) < tol, \
        f"Backward Pass Failed: Custom Grad: {xat.grad.item()}, PyTorch Grad: {xpt.grad.item()}"

    return "Sanity Check Passed!"

In [4]:
def test_more_ops():
    """
    Test a wider variety of operations, including division, exponentiation, and multiple accumulations.
    """
    a = Tensor(np.array([-4.0]))
    b = Tensor(np.array([2.0]))
    c = a + b
    d = a * b + b**3
    c = c + c + 1
    c = c + 1 + c + (-a)
    d = d + d * 2 + (b + a).relu()
    d = d + 3 * d + (b - a).relu()
    e = c - d
    f = e**2
    g = f / 2.0
    g = g + 10.0 * (f**-1)
    g.backward()
    aat, bat, gat = a, b, g



    a = torch.Tensor([-4.0]).double()
    b = torch.Tensor([2.0]).double()
    a.requires_grad = True
    b.requires_grad = True
    c = a + b
    d = a * b + b**3
    c = c + c + 1
    c = c + 1 + c + (-a)
    d = d + d * 2 + F.leaky_relu(b + a, negative_slope=0.01)
    d = d + 3 * d + F.leaky_relu(b - a, negative_slope=0.01)
    e = c - d
    f = e**2
    g = f / 2.0
    g = g + 10.0 * (f**-1)
    g.backward()
    apt, bpt, gpt = a, b, g

    tol = 1e-6


    assert abs(gat.data.item() - gpt.data.item()) < tol, \
        f"Forward Pass Failed: Custom: {gat.data.item()}, PyTorch: {gpt.data.item()}"
    assert abs(aat.grad.item() - apt.grad.item()) < tol, \
        f"Backward Pass (a) Failed: Custom Grad: {aat.grad.item()}, PyTorch Grad: {apt.grad.item()}"
    assert abs(bat.grad.item() - bpt.grad.item()) < tol, \
        f"Backward Pass (b) Failed: Custom Grad: {bat.grad.item()}, PyTorch Grad: {bpt.grad.item()}"

    return "More Ops Check Passed!"

In [5]:
try:
    result = test_sanity_check()
    print(f"✅ {result}")
except AssertionError as e:
    print(f"❌ Sanity Check FAILED: {e}")

try:
    result = test_more_ops()
    print(f"✅ {result}")
except AssertionError as e:
    print(f"❌ More Ops Check FAILED: {e}")

✅ Sanity Check Passed!
✅ More Ops Check Passed!
